In [1]:
!pip3 install pytorch-pretrained-biggan

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [6]:
import numpy as np, pandas as pd, os
import os, torch, gc
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_pretrained_biggan import BigGAN, truncated_noise_sample

# LOAD PRETRAINED BIGGAN-DEEP-128
model = BigGAN.from_pretrained('biggan-deep-256')

In [15]:
indexs=[]
tmp=range(151,275)
for x in tmp:
    indexs.append(x)
indexs.remove(251)
indexs.remove(269)
indexs.remove(270)
indexs.remove(271)
indexs.remove(272)
indexs.append(275)

In [16]:
print(indexs)

[151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 273, 274]


In [33]:

os.mkdir('./dataset_augmentated')
os.mkdir('./dataset_augmentated/images/')
SAMPLES = 25
iteration=10 #samples*iteration=quante img per classe
cont=0
for class_ind in indexs:
    # TRUNCATION VARIABLE AFFECTS QUALITY AND DIVERSITY
    truncation=0.9
    os.mkdir('./dataset_augmentated/images/'+str(class_ind))
    for i in range(iteration): # 25 alla volta per 10 volte,per evitare di andare cuda out of memory
        # GENERATE SAMPLES
        #truncation=np.random.randint(3,11)
        #truncation=truncation/10
        class_vector = np.zeros((SAMPLES,1000),dtype=np.float32)
        for x in range(SAMPLES): 
            class_vector[x,class_ind]=1 #(281,294) are cats (151,281) dog
            # CATEGORY LIST HERE: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
        noise_vector = truncated_noise_sample(truncation=truncation, batch_size=SAMPLES)
        noise_vector = torch.from_numpy(noise_vector)
        class_vector = torch.from_numpy(class_vector)

        # If you have a GPU, put everything on cuda
        noise_vector = noise_vector.to('cuda')
        class_vector = class_vector.to('cuda')
        model.to('cuda')

        # Generate an image
        with torch.no_grad():
            output = model(noise_vector, class_vector, truncation)

            # If you have a GPU put back on CPU
        output = output.to('cpu')
        output = output.numpy().transpose(0, 2, 3, 1)
        for j in range(SAMPLES):
            cont+=1
            img = Image.fromarray( ((output[j,]+1.)/2.*255).astype('uint8') )
            img = img.resize((256,256), Image.ANTIALIAS)
            img.save('./dataset_augmentated/images/'+str(class_ind)+'/'+str(cont)+'.png','PNG')

In [30]:
#clean old dataset


In [58]:
stock_dir_name=os.listdir('./dataset_clean/images')
cleaned_name=[]
for name in stock_dir_name:
    name=name.split('-',1)[1:][0]
    cleaned_name.append(name)


for name,clean in zip(stock_dir_name,cleaned_name):
    os.rename('./dataset_clean/images/'+name,'./dataset_clean/images/'+clean)

In [59]:
#rename new dataset

In [62]:
dir_name=os.listdir('./dataset_augmentated/images')
print(dir_name)

for name,clean in zip(dir_name,cleaned_name):
    os.rename('./dataset_augmentated/images/'+name,'./dataset_augmentated/images/'+clean)

    

['151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '273', '274', '275']
